**Database Systems 2017 (ITWS-6960)**

**Final Project**

**Stephan Zednik**

In [1]:
import re
import pandas
import matplotlib

In [2]:
%matplotlib inline

# Recipe Data

In [3]:
from recipes import RecipeData

Open a connection to the recipes database

In [4]:
recipes = RecipeData()
recipes.is_connected()

True

Use the ``RecipeData`` fluent interface to query recipe data from MongoDB

*query - find 5 recipes for alcoholic eggnog that are under 400 calories*

In [5]:
recipes.query().title_contains("eggnog").category("alcoholic").max_calories(400).run().head(5)

,_id,calories,categories,date,desc,directions,fat,ingredients,protein,rating,sodium,title
0,5a232c6de6136c4edf871886,264.0,"[Milk/Cream, Egg Nog, Non-Alcoholic, Egg, Chri...",2004-08-20T20:58:12.000Z,None,[Beat the egg with the sugar and salt and pour...,12.0,"[1 whole egg, 1 tablespoon superfine sugar, Pi...",12.0,2.50,450.0,Eggnog
1,5a232c6de6136c4edf8725a5,203.0,"[Bourbon, Liqueur, Milk/Cream, Rum, Alcoholic,...",2004-08-20T20:58:12.000Z,None,[Beat the egg yolks until they are light and g...,8.0,"[12 egg yolks, 1 cup fine granulated sugar, Cr...",1.0,0.00,18.0,Eggnoggin
2,5a232c6de6136c4edf8737da,378.0,"[Milk/Cream, Rum, Egg Nog, Alcoholic, Egg, Chr...",2004-08-20T20:58:12.000Z,None,[Beat egg yolks until they are thick and pale ...,29.0,"[12 egg yolks, 2 1/2 cups sugar, 3 quarts heav...",2.0,0.00,31.0,Frozen Eggnog
3,5a232c6de6136c4edf87393f,232.0,"[Milk/Cream, Rum, Egg Nog, Alcoholic, Egg, Chr...",2004-08-20T20:58:12.000Z,None,[Beat the egg yolks and sugar together until t...,9.0,"[12 eggs, separated, 2 cups superfine sugar, 1...",4.0,5.00,52.0,Baltimore Eggnog
4,5a232c6de6136c4edf873c35,296.0,"[Milk/Cream, Rum, Alcoholic, Mixer, Egg, Chris...",2004-08-20T20:58:12.000Z,None,"[In the bowl of an electric mixer, dissolve th...",8.0,"[1 cup honey, 1 cup warm water, 1 cup light ru...",2.0,3.75,26.0,Honey Eggnog


The ``.show()`` method can be used with the fluent interface to show what the compiled pymongo query looks like

In [6]:
recipes.query().title_contains("eggnog").category("alcoholic").max_calories(400).show()

{'calories': {'$lte': 400},
 'categories': re.compile(r'alcoholic', re.IGNORECASE|re.UNICODE),
 'title': re.compile(r'eggnog', re.IGNORECASE|re.UNICODE)}

# Product Data

In [7]:
from products import ProductData

Open a connection to the products database

In [8]:
products = ProductData("host='localhost' dbname='foodfacts' user='foodfacts' password='foodfacts'")

Use the ``ProductData`` fluent interface to query for product data from the PostgreSQL db

*query - find Kroger-brand products whose name includes 'Enriched Flour'*

In [9]:
products.query().name_includes("Enriched Flour").brand("Kroger").run()

,allergens,brands,calories_100g,calories_from_fat_100g,carbohydrates_100g,code,fat_100g,name,protein_100g,saturated_fat_100g,sugars_100g,url
0,{},Kroger,1393.0,None,73.33,11110854018,0.00,Bleached All Purpose Enriched Flour,10.00,0.00,3.33,http://world-en.openfoodfacts.org/product/0011...
1,{},Kroger,1393.0,None,73.33,11110861160,0.00,Unbleached All Purpose Enriched Flour,10.00,0.00,3.33,http://world-en.openfoodfacts.org/product/0011...


As with the ``RecipeData`` API, the ``.show()`` method can be used to show the SQL query that would be executed against the PostgreSQL db for a given query construction.

In [10]:
products.query().name_includes("Enriched Flour").brand("Kroger").show()

b"SELECT DISTINCT products.* from products WHERE LOWER(products.name) like LOWER('%Enriched Flour%') AND LOWER(brands) like LOWER('%Kroger%') ORDER BY products.code"

The fluent API supports some method being called multiple times so a query can indicate that more than 1 category for a given product should be matched against.  This is treated as an AND over the constraints.  

The ProductData API does not currently support the ability to provide OR over multiple values, but this could be added at a later time.

In [11]:
products.query().category("alcoholic").category("christmas").max_calories(500).show()

b"SELECT DISTINCT products.* from products JOIN product_category as product_category0 on products.code = product_category0.product JOIN categories as categories0 on categories0.name = product_category0.category JOIN product_category as product_category1 on products.code = product_category1.product JOIN categories as categories1 on categories1.name = product_category1.category WHERE LOWER(categories0.name) like LOWER('%alcoholic%') AND LOWER(categories1.name) like LOWER('%christmas%') AND products.calories_100g <= 500 ORDER BY products.code"

In [12]:
products.query().category("alcoholic").category("christmas").max_calories(500).run()

,allergens,brands,calories_100g,calories_from_fat_100g,carbohydrates_100g,code,fat_100g,name,protein_100g,saturated_fat_100g,sugars_100g,url
0,{},Leffe,239.0,None,5.30,5410228201144,0.00,Bière de Noël,0.50,0.00,0.30,http://world-en.openfoodfacts.org/product/5410...


## SQL Injection

The [psycopg2](http://initd.org/psycopg/docs/usage.html#passing-parameters-to-sql-queries) library is used to escape query parameters for the purpose of defending against SQL-injection attacks.

*Test resistence to SQL-injection*

In [13]:
products.query().name_includes("'); SELECT * from products;").show()

b"SELECT DISTINCT products.* from products WHERE LOWER(products.name) like LOWER('%''); SELECT * from products;%') ORDER BY products.code"

The ``'`` of the injection is escaped so the entire injection attack is treated as a literal value instead of as part of a DQL (SELECT), DML (UPDATE, DELETE), or DDL (DROP) command

In [14]:
products.query().name_includes("'); SELECT * from products;").run()

""


In [15]:
products.query().brand("); DROP products; COMMIT; ").run()

""


note - I am a little surprised that the resulting DataFrame does not have columns (something I may look into later...) but a follow-up query over products will how that the ``DROP products;`` query was not sucessful.

In [16]:
# find all products that include peanuts
products.query().ingredient("peanut").brand("Kroger").run()

,allergens,brands,calories_100g,calories_from_fat_100g,carbohydrates_100g,code,fat_100g,name,protein_100g,saturated_fat_100g,sugars_100g,url
0,{},Kroger,None,None,21.43,01110843432,53.39,"Mixed Nuts, Salted",None,7.14,None,http://world-en.openfoodfacts.org/product/0111...
1,{},"Simple Truth, The Kroger Co.",2690.0,None,17.86,011110024567,53.57,"Roasted Almonds, Sea Salt",21.43,3.57,3.57,http://world-en.openfoodfacts.org/product/0111...
2,{},Kroger,916.0,None,26.56,011110560601,10.94,Party Pail Ice Cream,3.12,6.25,18.75,http://world-en.openfoodfacts.org/product/0111...
3,{},Kroger,2213.0,None,32.35,11110004925,38.24,Roasted Peanuts And Honey,17.65,7.35,20.59,http://world-en.openfoodfacts.org/product/0011...
4,{},Kroger,2389.0,None,28.57,11110008367,46.43,Salted Cashews Halves & Pieces,17.86,8.93,3.57,http://world-en.openfoodfacts.org/product/0011...
5,{},Kroger,2389.0,None,28.57,11110008398,46.43,"Halves & Pieces Cashews, Lightly Salted",17.86,8.93,3.57,http://world-en.openfoodfacts.org/product/0011...
6,{},Kroger,2389.0,None,28.57,11110008527,50.00,Unsalted Whole Cashews,17.86,8.93,3.57,http://world-en.openfoodfacts.org/product/0011...
7,{},Kroger,2389.0,None,28.57,11110008565,46.43,Salted Cashews Halves & Pieces,17.86,8.93,3.57,http://world-en.openfoodfacts.org/product/0011...
8,{},Kroger,2540.0,None,17.86,11110008626,53.57,Salted Mixed Nuts,21.43,7.14,3.57,http://world-en.openfoodfacts.org/product/0011...
9,{},Kroger,2540.0,None,17.86,11110008688,53.57,Salted Mixed Nuts,21.43,7.14,3.57,http://world-en.openfoodfacts.org/product/0011...


# Distributed Queries - finding products based on recipe ingredients

I would like to run a query to find potential products to use as ingredients in a recipe.  To implement this query I will have to run a query for recipe data against the PostgreSQL database and a query for product data against the MongoDB database and then join the results of the two queries.

I will write a ``DistributedQuery`` class that provides a fluent interface that allows the user to construct queries to run against the products and recipes databases and then performs a join on recipes.ingredient to product.name and returns a joined panda DataFrame.

*example - find kroger-brand products that can be used as ingredients in highly-rated (4+) brunch recipes that contain eggs and bacon and the recipe is under 500 calories*
```
DistributedQuery(products, recipes)\
    .recipe_category("brunch")\
    .recipe_ingredients_contains_all_of(["bacon", "egg"])\
    .recipe_min_rating(4)\
    .recipe_max_calories(500)\
    .product_brand("Kroger")\
    .run()
```

To complicate matters, the recipe ingredients array and product name fields do not join nicely.  The ingredient field often contains amount and other descriptive information that makes it difficult to match against product names with high degrees of confidence.  The product name fields also occasionally include brand and other descriptive information that make it difficult to automatically match against a generic ingredient name.  To perform the join I will have to break apart the ingredient and product name fields, join on subsets of the field and then compute a weight value to provide a simple estimation of the confidence of the match.  If my key matching criteria is too strict it will miss products that make sense for an ingredient need, if the key matching criteria is too loose there will be a very large number of false matches.

I will compute weights on the joins and select (by trial and error) a reasonable default minimum weight to use to filter out low-confidence joins.  I will also add a method to the fluent interface ``.min_weight(number)`` that will allow the user to adjust the weight value used to filter out low-confidence joins.

In [17]:
class DistributedQuery:
    
    '''Provides a fluent API to construct and run queries against the products and recipe databases 
    and does a 'fuzzy' join of the query results on (recipes.ingredients, products.name)'''
    
    def __init__(self, products, recipes):
        self.products_query = products.query()
        self.recipes_query = recipes.query()
        self.min_merge_weight = 0.2
    
    @staticmethod
    def get_text_breakdown(ingredient):
                
        if not ingredient:
            return []
        
        ingredient = ingredient.strip()#.split(',')[0]
        _last = len(ingredient)
        run = True
        parts = []

        while run:
            idx = ingredient.rfind(" ", 0, _last)
            _last = idx

            if idx == -1:
                parts.append(ingredient.lower())
                run = False
            else:
                parts.append(ingredient[idx:].strip(" ,()-").lower())

        return parts
    
    def min_weight(self, weight):
        self.min_merge_weight = weight
        return self
    
    def product_category(self, category):
        self.products_query = self.products_query.category(category)
        return self
    
    def product_max_calories(self, max_calories):
        self.products_query = self.products_query.max_calories(max_calories)
        return self
    
    def product_brand(self, brand):
        self.products_query = self.products_query.brand(brand)
        return self
    
    def recipe_max_calories(self, max_calories):
        self.recipes_query = self.recipes_query.max_calories(max_calories)
        return self
    
    def recipe_min_rating(self, min_rating):
        self.recipes_query = self.recipes_query.min_rating(min_rating)
        return self
    
    def recipe_ingredients_contains_all_of(self, ingredients):
        self.recipes_query = self.recipes_query.ingredient_all_of(ingredients)
        return self
    
    def recipe_category(self, category):
        self.recipes_query = self.recipes_query.category(category)
        return self
    
    def recipe_category_any_of(self, categories):
        self.recipes_query = self.recipes_query.category_any_of(categories)
        return self
    
    def recipe_category_all_of(self, categories):
        self.recipes_query = self.recipes_query.category_all_of(categories)
        return self
    
    @staticmethod
    def products_add_name_key(df):
        df = df.add_prefix("product_")
        rows = []
        _ = df.apply(lambda row: [rows.append(row.tolist()+[name_part]) for name_part in DistributedQuery.get_text_breakdown(row["product_name"])], axis=1)
        return pandas.DataFrame(rows, columns=df.columns.tolist()+['name_key']).drop_duplicates()
    
    @staticmethod
    def recipes_add_ingredient_key(df):
        df = df.add_prefix("recipe_")

        rows = []
        _ = df.apply(lambda row: [rows.append(row.tolist()+[ingredient]) for ingredient in row["recipe_ingredients"]], axis=1)
        _df = pandas.DataFrame(rows, columns=df.columns.tolist()+['recipe_ingredient']).drop_duplicates(['recipe_title','recipe_ingredient'])

        rows = []
        for index, row in _df.iterrows():
            for part in DistributedQuery.get_text_breakdown(row["recipe_ingredient"]):
                rows.append(row.tolist()+[part])
                
        return pandas.DataFrame(rows, columns=_df.columns.tolist()+['ingredient_key']).drop_duplicates(['recipe_title', 'recipe_ingredient', 'ingredient_key'])    
    
    @staticmethod
    def compute_weight(key, field):
        
        if not key or not field:
            return 0
        
        if pandas.isnull(key) or pandas.isnull(field):
            return 0
        
        return len(key)/len(field)
    
    @staticmethod
    def compute_merge_weight(df):
        df['ingredient_key_weight'] = df.apply(lambda row: DistributedQuery.compute_weight(row['ingredient_key'], row['product_name']), axis=1)
        df['name_key_weight'] = df.apply(lambda row: DistributedQuery.compute_weight(row['name_key'], row['recipe_ingredient']), axis=1)        
        return df
    
    @staticmethod
    def merge_recipes_and_products(df_recipes, df_products):        
        df_recipes2 = DistributedQuery.recipes_add_ingredient_key(df_recipes)
        df_products2 = DistributedQuery.products_add_name_key(df_products)
        
        df = pandas.merge(df_recipes2, df_products2, how='left', left_on="ingredient_key", right_on="name_key", suffixes=('_recipe','_product'))
        df = DistributedQuery.compute_merge_weight(df)
        df.drop('ingredient_key', axis=1, inplace=True)
        df.drop('name_key', axis=1, inplace=True)
        df = df.drop_duplicates(['recipe_title', 'recipe_ingredient', 'product_brands', 'product_name'])
        return df
    
    def run(self):
        '''run the queries, merge via 'fuzzy join' and return the merged result as a pandas DataFrame '''
        df_recipes = self.recipes_query.run()
        df_products = self.products_query.run()
        df_merged = self.merge_recipes_and_products(df_recipes, df_products)
        
        if self.min_merge_weight > 0:
            df_merged = df_merged.loc[(df_merged['ingredient_key_weight'] > self.min_merge_weight) 
                                      & (df_merged['name_key_weight'] > self.min_merge_weight)]
        
        return df_merged
    
    def show(self):
        '''return a dictionary showing the recipe and product queries that would be run and merged'''
        return {"products": self.products_query.show(), "recipes": self.recipes_query.show()}

default_columns = ['recipe_title', 'recipe_ingredient', 'recipe_rating', 'recipe_calories', 'product_name', 'product_brands', 'ingredient_key_weight', 'name_key_weight']

def filter_by_recipe_title(df, recipe, columns=default_columns):
    '''helper function to easily filter query results by a recipe name'''
    return df.loc[df['recipe_title'].str.contains(recipe)][columns]

def simplify(df, columns=default_columns):
    '''helper function to reduce data frame to a subset of columns'''
    return df[columns]

*query - find kroger-brand products that can be used as ingredients in highly-rated (4+) brunch recipes that contain eggs and bacon and for which recipe is under 500 calories*

In [18]:
%time df = DistributedQuery(products, recipes)\
    .recipe_category("brunch")\
    .recipe_ingredients_contains_all_of(["bacon", "egg"])\
    .recipe_min_rating(4)\
    .recipe_max_calories(500)\
    .product_brand("Kroger")\
    .run()

CPU times: user 397 ms, sys: 9.47 ms, total: 406 ms
Wall time: 536 ms


In [19]:
simplify(df)

,recipe_title,recipe_ingredient,recipe_rating,recipe_calories,product_name,product_brands,ingredient_key_weight,name_key_weight
0,Buckwheat Bacon Pancakes,1/2 cup buckwheat flour,4.375,269.0,Whole Wheat Flour,Kroger,0.294118,0.217391
3,Buckwheat Bacon Pancakes,1/2 cup buckwheat flour,4.375,269.0,Coconut Flour,The Kroger Co.,0.384615,0.217391
15,Buckwheat Bacon Pancakes,1 teaspoon sugar,4.375,269.0,Dark Brown Sugar,Kroger,0.312500,0.312500
16,Buckwheat Bacon Pancakes,1 teaspoon sugar,4.375,269.0,Light Brown Sugar,The Kroger Co.,0.294118,0.312500
19,Buckwheat Bacon Pancakes,1 teaspoon baking powder,4.375,269.0,Chili Powder,Kroger,0.500000,0.250000
22,Buckwheat Bacon Pancakes,1 teaspoon baking powder,4.375,269.0,Baking Powder,Kroger,0.461538,0.250000
35,Buckwheat Bacon Pancakes,1/4 teaspoon salt,4.375,269.0,Celery Salt,Kroger,0.363636,0.235294
36,Buckwheat Bacon Pancakes,1/4 teaspoon salt,4.375,269.0,Onion Salt,Kroger,0.400000,0.235294
37,Buckwheat Bacon Pancakes,1/4 teaspoon salt,4.375,269.0,Seasoned Salt,Kroger,0.307692,0.235294
38,Buckwheat Bacon Pancakes,1/4 teaspoon salt,4.375,269.0,Garlic Salt,Kroger,0.363636,0.235294


*Re-run the query, this time looking for Safeway-brand products*

In [20]:
%time df2 = DistributedQuery(products, recipes)\
    .recipe_category("brunch")\
    .recipe_ingredients_contains_all_of(["bacon", "egg"])\
    .recipe_min_rating(4)\
    .recipe_max_calories(500)\
    .product_brand("Safeway")\
    .run()

CPU times: user 182 ms, sys: 3.38 ms, total: 185 ms
Wall time: 305 ms


In [21]:
simplify(df2)

,recipe_title,recipe_ingredient,recipe_rating,recipe_calories,product_name,product_brands,ingredient_key_weight,name_key_weight
0,Buckwheat Bacon Pancakes,1/2 cup buckwheat flour,4.375,269.0,All Purpose Flour,Safeway Inc.,0.294118,0.217391
10,Buckwheat Bacon Pancakes,1 teaspoon sugar,4.375,269.0,Light Brown Sugar,"Signature Kitchens, Safeway Inc.",0.294118,0.312500
11,Buckwheat Bacon Pancakes,1 teaspoon sugar,4.375,269.0,Dark Brown Sugar,Safeway Inc.,0.312500,0.312500
14,Buckwheat Bacon Pancakes,1 teaspoon baking powder,4.375,269.0,Chili Powder,Safeway,0.500000,0.250000
46,Buckwheat Bacon Pancakes,1 1/4 cups well-shaken buttermilk,4.375,269.0,Lowfat Cultured Buttermilk,Safeway Inc.,0.384615,0.303030
48,Buckwheat Bacon Pancakes,1 1/4 cups well-shaken buttermilk,4.375,269.0,"Enriched Bread, Country Buttermilk","Signature Kitchens, Safeway Inc.",0.294118,0.303030
49,Buckwheat Bacon Pancakes,1 1/4 cups well-shaken buttermilk,4.375,269.0,"Enriched Wheat Bread, Buttermilk","Signature Kitchens, Safeway Inc.",0.312500,0.303030
89,Creamy Grits with Rosemary Bacon,8 thick-cut bacon slices,4.375,497.0,Dried Philippine Mango Slices,"Safeway Select, Safeway Inc.",0.206897,0.250000
96,Creamy Grits with Rosemary Bacon,8 thick-cut bacon slices,4.375,497.0,Assorted Fruit Slices,"Safeway, Safeway Inc.",0.285714,0.250000
97,Creamy Grits with Rosemary Bacon,8 thick-cut bacon slices,4.375,497.0,"Candy, Watermelon Slices","Signature Kitchens, Safeway Inc.",0.250000,0.250000


try the product, recipe, and distributedQuery interfaces yourself!

examples:
- ``products.query().name_includes("honey").brand("Trader Joe's").max_sugar(75).run()``
- ``recipes.query().category_any_of(["easter", "christmas", "thanksgiving"]).min_rating(4).run()``
- ``DistributedQuery(products, recipes)\
.recipe_category("christmas")\
.recipe_min_rating(4)\
.recipe_max_calories(500)\
.product_brand("Trader Joe's")\
.run()``